In [3]:

import soundfile as sf
import csv
import time
import threading
import sounddevice as sd
import numpy as np
from pynput import keyboard
from scipy.io.wavfile import write as wav_write

### The following code is used to record the audio and the keystrokes

*It is not used in the final project, but it is kept here for reference
*The audio recording will go on for 10 seconds. The keystroke recording will be done simultaneously but needs to be stopped manually by pressing ESC (escape)

Head's up : Dont forget to grant accessibility access the editor with which you're running the code below, in particular input monitoring for keystroke recording.


In [7]:
import numpy as np
import csv
import time
import sounddevice as sd
import threading
import scipy.io.wavfile as wav
from pynput import keyboard

# Parameters for sound recording
sample_rate = 44100  # Hz
duration = 10  # seconds
audio_data = np.zeros((duration * sample_rate, 1), dtype=np.int16)

# File paths
audio_file = 'keyboard_sound.wav'
log_file = 'key_log.csv'

# Initialize the keystroke log file
with open(log_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Key', 'Action', 'Timestamp'])

# Global variable to track the start time of audio recording
start_time_audio = None
stop_recording = False  # Flag to stop both audio and keyboard listeners

# Recording function for audio
def record_audio():
    global audio_data, start_time_audio
    print("Recording audio...")

    # Set the start time of the recording
    start_time_audio = time.time()  # Capture the timestamp when recording starts

    # Record the audio and store it in audio_data
    try : 
        audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=2, dtype='int16')
        sd.wait()
        wav.write(audio_file, sample_rate, audio_data)
        print(f"Audio saved to {audio_file}")
    except:
        print("Your computer does not support stereo recording. Defaulting to mono.")
        audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
        sd.wait()
        wav.write(audio_file, sample_rate, audio_data)
        print(f"Audio saved to {audio_file}")

    print("Audio recording finished")

    # Save the recorded audio to a WAV file

# Keystroke listener function
def on_press(key):
    if start_time_audio is None:
        return  # Don't log if the audio hasn't started yet

    timestamp = time.time() - start_time_audio  # Calculate relative timestamp
    try:
        key_str = key.char  # Normal keys
    except AttributeError:
        key_str = str(key)  # Special keys like shift, ctrl, etc.

    # Log the key press with relative timestamp
    with open(log_file, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([key_str, 'Pressed', round(timestamp, 6)])  # Round for cleaner timestamp

    print(f"Key {key_str} Pressed at {timestamp:.6f} seconds")

def on_release(key):
    if start_time_audio is None:
        return  # Don't log if the audio hasn't started yet

    timestamp = time.time() - start_time_audio  # Calculate relative timestamp
    try:
        key_str = key.char
    except AttributeError:
        key_str = str(key)

    # Log the key release with relative timestamp
    with open(log_file, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([key_str, 'Released', round(timestamp, 6)])  # Round for cleaner timestamp

    print(f"Key {key_str} Released at {timestamp:.6f} seconds")

    # Stop listener if 'Esc' key is pressed
    if key == keyboard.Key.esc:
        global stop_recording
        stop_recording = True  # Set flag to stop both recordings
        return False

# Start recording audio in a separate thread
audio_thread = threading.Thread(target=record_audio)
audio_thread.start()

# Start the keyboard listener in the main thread to avoid blocking
def start_keyboard_listener():
    with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
        listener.join()

# Run the keyboard listener in a separate thread
keyboard_thread = threading.Thread(target=start_keyboard_listener)
keyboard_thread.start()

# Wait for both threads to finish, while checking for stop condition
while not stop_recording:
    time.sleep(0.1)  # Prevent high CPU usage by sleeping briefly

# Once 'Esc' is pressed, both threads will finish
audio_thread.join()
keyboard_thread.join()
print("Recording process finished.")

This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


Recording audio...
Your computer does not support stereo recording. Defaulting to mono.
Key h Pressed at 1.493454 seconds
Key h Released at 1.617332 seconds
Key g Pressed at 2.057544 seconds
Key g Released at 2.206375 seconds
Key l Pressed at 2.835166 seconds
Key l Released at 2.980438 seconds
Key o Pressed at 3.294037 seconds
Key o Released at 3.430136 seconds
Key q Pressed at 3.857064 seconds
Key q Released at 4.041033 seconds
Key e Pressed at 4.394897 seconds
Key e Released at 4.519266 seconds
Key r Pressed at 4.804373 seconds
Key r Released at 4.948341 seconds
Key t Pressed at 5.258346 seconds
Key t Released at 5.414851 seconds
Key y Pressed at 5.735761 seconds
Key y Released at 5.895222 seconds
Key u Pressed at 6.177737 seconds
Key u Released at 6.327976 seconds
Key i Pressed at 6.610898 seconds
Key i Released at 6.731015 seconds
Key k Pressed at 7.061621 seconds
Key k Released at 7.221347 seconds
Key s Pressed at 7.571879 seconds
Key s Released at 7.772534 seconds
Key h Pressed a

In [7]:
print(sd.query_devices())

> 0 MacBook Pro Microphone, Core Audio (1 in, 0 out)
< 1 MacBook Pro Speakers, Core Audio (0 in, 2 out)
  2 Microsoft Teams Audio, Core Audio (1 in, 1 out)


In [8]:
# Load the audio file
audio_path = "keyboard_sound.wav"
audio_data, sample_rate = sf.read(audio_path)

# Read keystroke timestamps from CSV
keystroke_times = []
with open("key_log.csv", "r") as audio_data_file:
    reader = csv.reader(audio_data_file)
    stack = {}

    for row in reader:
        if row[0] == "Key":
            continue
        key = row[0]
        action = row[1]
        timestamp = float(row[2])

        if action == "Pressed":
            if key in stack:
                raise Exception(f"Key {key} was pressed again before being released.")
            stack[key] = [key, timestamp]

        elif action == "Released":
            if key not in stack:
                raise Exception(f"Key {key} was released without being pressed.")
            stack[key].append(timestamp)
            keystroke_times.append(stack[key])
            del stack[key]

print(keystroke_times)


[['h', 1.493454, 1.617332], ['g', 2.057544, 2.206375], ['l', 2.835166, 2.980438], ['o', 3.294037, 3.430136], ['q', 3.857064, 4.041033], ['e', 4.394897, 4.519266], ['r', 4.804373, 4.948341], ['t', 5.258346, 5.414851], ['y', 5.735761, 5.895222], ['u', 6.177737, 6.327976], ['i', 6.610898, 6.731015], ['k', 7.061621, 7.221347], ['s', 7.571879, 7.772534], ['h', 8.090684, 8.259104], ['f', 8.557782, 8.702709], ['j', 8.94284, 9.106668], ['t', 9.584076, 9.69771], ['h', 9.989638, 10.159746], ['b', 10.469989, 10.616669], ['Key.esc', 11.03878, 11.122985]]


## The code that generates individual spectrograms for each keystroke

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
import scipy.signal as signal
import csv
import os

# Parameters
AUDIO_FILE = "keyboard_sound.wav"  # Replace with your actual audio file
KEYSTROKE_CSV = "key_log.csv"
OUTPUT_DIR = "keystroke_spectrograms/keystroke_spectrograms"
NUMPY_OUTPUT_DIR = "keystroke_spectrograms/numpy_arrays"  # New directory for NumPy arrays

## Hyperparameter to fine-tune
BUFFER_TIME = 0.2  # Extra time (seconds) before & after each keystroke

# Ensure output directories exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(NUMPY_OUTPUT_DIR, exist_ok=True)

# Load audio data
sample_rate, audio_data = wav.read(AUDIO_FILE)

# Read keystroke data from CSV
keystroke_times = []
with open(KEYSTROKE_CSV, "r") as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    stack = {}

    for row in reader:
        key, action, timestamp = row[0], row[1], float(row[2])

        if action == "Pressed":
            stack[key] = timestamp  # Store press time
        elif action == "Released" and key in stack:
            keystroke_times.append((key, stack.pop(key), timestamp))  # Store key, press, and release

# Function to create and save the spectrogram and numpy arrays
def create_spectrogram_and_numpy(audio_segment, key, idx):
    # Generate the spectrogram using scipy
    f, t, Sxx = signal.spectrogram(audio_segment, sample_rate)

    # Normalize spectrogram for neural network (optional)
    Sxx_log = 10 * np.log10(Sxx + 1e-10)  # Adding small constant to avoid log(0)

    # Plot the spectrogram
    plt.figure(figsize=(10, 4))
    plt.pcolormesh(t, f, Sxx_log, shading='auto', cmap='inferno')
    plt.colorbar(label='Power (dB)')
    plt.title(f"Keystroke '{key}' Spectrogram")
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")

    # Save the spectrogram to file as PNG
    spectrogram_path = os.path.join(OUTPUT_DIR, f"keystroke_{idx + 1}_{key}.png")
    plt.savefig(spectrogram_path)
    plt.close()
    print(f"Saved spectrogram for '{key}' at {spectrogram_path}")

    # Save the spectrogram as a NumPy array
    numpy_array_path = os.path.join(NUMPY_OUTPUT_DIR, f"keystroke_{idx + 1}_{key}.npy")
    np.save(numpy_array_path, Sxx_log)  # Save the log-scaled spectrogram as a numpy array
    print(f"Saved NumPy array for '{key}' at {numpy_array_path}")

# Process keystrokes
for idx, (key, press_time, release_time) in enumerate(keystroke_times):
    # Define start and end times (add buffer)
    start_time = max(0, press_time - BUFFER_TIME)
    end_time = min(len(audio_data) / sample_rate, release_time + BUFFER_TIME)

    # Convert times to sample indices
    start_sample = int(start_time * sample_rate)
    end_sample = int(end_time * sample_rate)

    # Extract audio segment
    keystroke_audio = audio_data[start_sample:end_sample]

    if len(keystroke_audio) == 0:
        print(f"Warning: Empty audio segment for keystroke {idx + 1}")
        continue

    # Create and save the spectrogram and numpy array for this key
    create_spectrogram_and_numpy(keystroke_audio, key, idx)

print("Processing complete. Spectrograms and NumPy arrays saved.")


Saved spectrogram for 'h' at keystroke_spectrograms/keystroke_spectrograms/keystroke_1_h.png
Saved NumPy array for 'h' at keystroke_spectrograms/numpy_arrays/keystroke_1_h.npy
Saved spectrogram for 'g' at keystroke_spectrograms/keystroke_spectrograms/keystroke_2_g.png
Saved NumPy array for 'g' at keystroke_spectrograms/numpy_arrays/keystroke_2_g.npy
Saved spectrogram for 'l' at keystroke_spectrograms/keystroke_spectrograms/keystroke_3_l.png
Saved NumPy array for 'l' at keystroke_spectrograms/numpy_arrays/keystroke_3_l.npy
Saved spectrogram for 'o' at keystroke_spectrograms/keystroke_spectrograms/keystroke_4_o.png
Saved NumPy array for 'o' at keystroke_spectrograms/numpy_arrays/keystroke_4_o.npy
Saved spectrogram for 'q' at keystroke_spectrograms/keystroke_spectrograms/keystroke_5_q.png
Saved NumPy array for 'q' at keystroke_spectrograms/numpy_arrays/keystroke_5_q.npy
Saved spectrogram for 'e' at keystroke_spectrograms/keystroke_spectrograms/keystroke_6_e.png
Saved NumPy array for 'e' a